# Exercises: Meshes

Build your understanding of the concepts from `core_04_meshes.ipynb`.

This notebook covers:
- **Triangular meshes** (`wp.Mesh`) and how Warp represents them
- **2D kernel launches** using `wp.tid()` with two dimensions
- **Signed Distance Fields (SDFs)** via `wp.mesh_query_point()`
- **Color maps** to visualize scalar fields
- **Raycasting** via `wp.mesh_query_ray()`

Each exercise has an empty code cell — fill it in, run it, and check the assertions.
If you get stuck, refer back to the tutorial notebook.

In [ ]:
import numpy as np
import warp as wp

wp.config.quiet = True
wp.init()

---
## Exercise 1: What Is a Triangular Mesh?

### Think first

A triangular mesh is the most common way to represent 3D surfaces in computer graphics
and simulation. It consists of:
- **Points** (vertices): a list of 3D positions
- **Indices** (faces): groups of 3 indices into the points array, each defining a triangle

For example, a single triangle uses 3 points and 3 indices `[0, 1, 2]`.
A square can be made from 4 points and 6 indices (two triangles sharing an edge).

**Question:** If you have 4 points forming a square in the XZ plane, how many triangles
do you need? How many index values is that?

<details>
<summary>Discussion</summary>

You need **2 triangles** to tile a square. Each triangle needs 3 index values,
so that's **6 index values** total. For example, with points at corners 0, 1, 2, 3,
you might use triangles `[0, 1, 2]` and `[0, 2, 3]`.
</details>

### Now write it

Create a `wp.Mesh` from a simple square (4 points, 2 triangles) lying in the XZ plane
at `y=0`, spanning from `(-1, 0, -1)` to `(1, 0, 1)`.

Warp meshes are created with:
```python
mesh = wp.Mesh(
    points=wp.array(..., dtype=wp.vec3),
    indices=wp.array(..., dtype=int),
)
```

In [ ]:
# Your code here.
#
# Define 4 points forming a square in the XZ plane (y=0).
# Define 6 indices (2 triangles) covering the square.
# Create a wp.Mesh and store it in a variable called `square_mesh`.


In [ ]:
# --- Assertions (do not modify) ---
assert square_mesh.points.shape == (4,), f"Expected 4 points, got shape {square_mesh.points.shape}"
assert square_mesh.indices.shape == (6,), f"Expected 6 indices, got shape {square_mesh.indices.shape}"
pts = square_mesh.points.numpy()
assert np.all(pts[:, 1] == 0.0), "All points should have y=0"
print(f"Mesh created: {square_mesh.points.shape[0]} points, {square_mesh.indices.shape[0] // 3} triangles")
print("Passed!")

---
## Exercise 2: 2D Kernel Launches

### Think first

So far you've launched kernels over 1D grids: `dim=N` gives you `i = wp.tid()`
ranging from 0 to N-1.

But for image/texture operations, a **2D grid** is natural. When you launch with
`dim=(width, height)`, `wp.tid()` returns **two** values:
```python
j, i = wp.tid()  # j = column (0..width-1), i = row (0..height-1)
```

**Important:** The first value corresponds to the first dimension (width/columns),
and the second to the second dimension (height/rows). When indexing a 2D array
stored as `[rows, cols]`, you typically write `array[i, j]` (row first).

**Question:** If you launch a kernel with `dim=(300, 200)`, how many threads run in total?
What range does the first `wp.tid()` value cover?

<details>
<summary>Discussion</summary>

Total threads = 300 * 200 = 60,000. The first value from `wp.tid()` ranges from
0 to 299, and the second from 0 to 199.
</details>

### Now write it

Write a kernel that fills a 2D array of `wp.vec3` with a color gradient.
Each pixel should be `wp.vec3(j / width, i / height, 0.0)` — so the red channel
increases left-to-right, and green increases top-to-bottom.

Use a `(16, 8)` grid (16 columns, 8 rows).

In [ ]:
# Your code here.
#
# Write a @wp.kernel called `fill_gradient` that takes:
#   width: float
#   height: float
#   out: wp.array(dtype=wp.vec3, ndim=2)
#
# Use j, i = wp.tid() and set out[i, j] to the gradient color.
# Then create a (8, 16) array (rows, cols) and launch with dim=(16, 8).
# Store the result array in a variable called `gradient`.


In [ ]:
# --- Assertions (do not modify) ---
g = gradient.numpy()
assert g.shape == (8, 16, 3), f"Expected shape (8, 16, 3), got {g.shape}"
# Top-left corner should be (0, 0, 0)
np.testing.assert_allclose(g[0, 0], [0.0, 0.0, 0.0], atol=0.01)
# Bottom-right corner should be close to (1, 1, 0) — but not exactly 1.0
# because we divide by width/height, giving (15/16, 7/8, 0)
np.testing.assert_allclose(g[7, 15], [15.0 / 16.0, 7.0 / 8.0, 0.0], atol=0.01)
print("Passed!")

---
## Exercise 3: Mesh Point Queries

### Think first

`wp.mesh_query_point()` is the core building block for collision detection and SDFs.
Given a mesh and a point in space, it finds the **closest location on the mesh surface**.

It returns a query result with:
- `result`: `bool` — whether a closest point was found
- `sign`: `float` — positive if the point is **outside** the mesh, negative if **inside**
- `face`: `int` — the triangle index containing the closest point
- `u`, `v`: `float` — barycentric coordinates within that triangle

You can then call `wp.mesh_eval_position(mesh_id, face, u, v)` to get the actual
3D position of the closest point.

**Question:** If you have a closed mesh (like a sphere) and a point inside it,
what sign would `query.sign` have? What about a point outside?

<details>
<summary>Discussion</summary>

A point **inside** the mesh gets `sign < 0` (negative), and a point **outside** gets
`sign >= 0` (positive). This is the "signed" in "Signed Distance Field" — the sign
tells you which side of the surface you're on.

Note: sign detection only works reliably for **closed** (watertight) meshes. For open
surfaces (like our flat square), the sign may not be meaningful.
</details>

### Now write it

Write a kernel that takes an array of query points and a mesh, and computes the
**unsigned distance** from each point to the closest surface location.

Use the square mesh from Exercise 1 (or create a new one). Query from 3 points:
1. `(0, 0, 0)` — on the surface (distance should be ~0)
2. `(0, 1, 0)` — directly above center (distance should be ~1)
3. `(0, 0.5, 0)` — halfway above center (distance should be ~0.5)

In [ ]:
# Your code here.
#
# Write a @wp.kernel called `compute_distance` that takes:
#   mesh_id: wp.uint64
#   query_points: wp.array(dtype=wp.vec3)
#   distances: wp.array(dtype=float)
#
# For each point:
#   1. Call wp.mesh_query_point(mesh_id, point, max_dist=1e6)
#   2. Check query.result
#   3. Use wp.mesh_eval_position(mesh_id, query.face, query.u, query.v)
#      to get the nearest surface position
#   4. Compute distance = wp.length(nearest_pos - point)
#   5. Store in distances[i]
#
# Create the square mesh (or reuse from Exercise 1), set up the 3 query points,
# and launch the kernel. Store distances in a variable called `dists`.


In [ ]:
# --- Assertions (do not modify) ---
d = dists.numpy()
print(f"Distances: {d}")
np.testing.assert_allclose(d[0], 0.0, atol=0.01, err_msg="Point on surface should have distance ~0")
np.testing.assert_allclose(d[1], 1.0, atol=0.01, err_msg="Point 1 unit above should have distance ~1")
np.testing.assert_allclose(d[2], 0.5, atol=0.01, err_msg="Point 0.5 units above should have distance ~0.5")
print("Passed!")

---
## Exercise 4: Signed Distance Fields (SDFs)

### Think first

A **Signed Distance Field** assigns a signed distance value to every point in space:
- **Negative** inside the mesh
- **Zero** on the surface
- **Positive** outside the mesh

SDFs are widely used for:
- **Collision detection**: check if `sdf < 0` to detect penetration
- **Rendering**: ray march through the field to find surfaces
- **Physics**: compute contact normals from the SDF gradient

To compute an SDF value from a mesh query, you combine the distance with the sign:
```python
dist = wp.length(nearest_pos - point)
sdf = dist if query.sign >= 0 else -dist
```

**Question:** For a sphere centered at the origin with radius 1, what SDF value
would the point `(0, 0, 2)` have? What about `(0, 0, 0.5)`?

<details>
<summary>Discussion</summary>

`(0, 0, 2)` is 1 unit outside the sphere surface, so SDF = +1.0.
`(0, 0, 0.5)` is 0.5 units inside the sphere surface, so SDF = -0.5.
The SDF of a sphere is simply `length(point) - radius`.
</details>

### Now write it

Create a closed mesh (a tetrahedron — the simplest closed 3D shape) and compute
the SDF at a few test points.

A regular tetrahedron centered at the origin with vertices at:
```
(1, 1, 1), (1, -1, -1), (-1, 1, -1), (-1, -1, 1)
```
has 4 triangular faces (12 indices). The faces with outward-pointing normals are:
```
[0, 1, 2], [0, 2, 3], [0, 3, 1], [1, 3, 2]
```

Write a kernel that computes the **signed** distance. Test with:
1. `(0, 0, 0)` — inside the tetrahedron (SDF should be negative)
2. `(3, 3, 3)` — outside the tetrahedron (SDF should be positive)

In [ ]:
# Your code here.
#
# 1. Create the tetrahedron mesh with the 4 vertices and 4 faces above.
#    Store as `tet_mesh`.
#
# 2. Write a @wp.kernel called `compute_sdf` that takes:
#      mesh_id: wp.uint64
#      query_points: wp.array(dtype=wp.vec3)
#      sdf_values: wp.array(dtype=float)
#
#    For each point, compute the signed distance using query.sign.
#    If sign >= 0, sdf = +dist. If sign < 0, sdf = -dist.
#
# 3. Launch with the 2 test points and store SDF values in `sdf_vals`.


In [ ]:
# --- Assertions (do not modify) ---
s = sdf_vals.numpy()
print(f"SDF values: {s}")
assert s[0] < 0.0, f"Origin should be inside (negative SDF), got {s[0]}"
assert s[1] > 0.0, f"(3,3,3) should be outside (positive SDF), got {s[1]}"
print("Passed!")

---
## Exercise 5: Color Maps

### Think first

A **color map** converts a scalar value (like an SDF distance) into an RGB color for
visualization. The tutorial uses a "Bourke color map" — a piecewise-linear function
that maps a range `[low, high]` through blue -> cyan -> green -> yellow -> red.

The idea is simple: divide the range into 4 equal segments and linearly interpolate
the R, G, B channels in each segment:

| Range segment | Color transition |
|---|---|
| 0% - 25% | Blue (0,0,1) -> Cyan (0,1,1) |
| 25% - 50% | Cyan (0,1,1) -> Green (0,1,0) |
| 50% - 75% | Green (0,1,0) -> Yellow (1,1,0) |
| 75% - 100% | Yellow (1,1,0) -> Red (1,0,0) |

**Question:** What color would a value exactly at the midpoint of the range produce?

<details>
<summary>Discussion</summary>

At the midpoint (50%), we're at the boundary between the cyan->green and green->yellow
segments. At exactly 50%, red is just starting to increase from 0, green is 1, and blue
is 0. So the color is green (0, 1, 0).
</details>

### Now write it

Implement the Bourke color map as a `@wp.func`. The function should:
1. Clamp `v` to `[low, high]`
2. Compute `dv = high - low`
3. Apply the 4-segment piecewise logic described above
4. Return `wp.vec3(r, g, b)`

Write a kernel that applies this to an array of float values and produces an array
of `wp.vec3` colors. Test with values at the extremes and midpoint.

In [ ]:
# Your code here.
#
# 1. Write a @wp.func called `bourke_color_map` that takes:
#      low: float, high: float, v: float -> wp.vec3
#
#    Start with r=1, g=1, b=1. Clamp v. Compute dv = high - low.
#    Then apply the 4 conditions:
#      if v < low + 0.25*dv:  r=0, g = 4*(v-low)/dv
#      elif v < low + 0.5*dv: r=0, b = 1 + 4*(low+0.25*dv-v)/dv
#      elif v < low + 0.75*dv: r = 4*(v-low-0.5*dv)/dv, b=0
#      else: g = 1 + 4*(low+0.75*dv-v)/dv, b=0
#
# 2. Write a @wp.kernel called `apply_color_map` that takes:
#      values: wp.array(dtype=float)
#      low: float
#      high: float
#      colors: wp.array(dtype=wp.vec3)
#
# 3. Test with values [-1.0, -0.5, 0.0, 0.5, 1.0] mapped over range [-1, 1].
#    Store the result in `colors`.


In [ ]:
# --- Assertions (do not modify) ---
c = colors.numpy()
print("Colors:")
for i, v in enumerate([-1.0, -0.5, 0.0, 0.5, 1.0]):
    print(f"  v={v:+.1f} -> RGB=({c[i, 0]:.2f}, {c[i, 1]:.2f}, {c[i, 2]:.2f})")
# At low end (-1.0): should be blue-ish (r~0, g~0, b~1)
assert c[0, 0] < 0.1, "At low end, red should be ~0"
assert c[0, 2] > 0.9, "At low end, blue should be ~1"
# At high end (1.0): should be red-ish (r~1, g~0, b~0)
assert c[4, 0] > 0.9, "At high end, red should be ~1"
assert c[4, 2] < 0.1, "At high end, blue should be ~0"
# At midpoint (0.0): should be green-ish
assert c[2, 1] > 0.9, "At midpoint, green should be ~1"
print("Passed!")

---
## Exercise 6: Rendering a 2D SDF Slice

### Think first

The tutorial renders a 2D "slice" of the mesh's SDF by:
1. Setting up a 2D grid of pixels in the XZ plane (at `y=0`)
2. For each pixel, computing its 3D position from its grid indices
3. Querying the mesh to get the signed distance
4. Mapping the SDF value to a color
5. Storing the color in a 2D array and displaying with matplotlib

The **pixel-to-world** mapping is:
```python
world_x = j * pixel_size + pixel_offset_x
world_z = i * pixel_size + pixel_offset_z
```

The **pixel offset** centers the grid at the origin and places queries at pixel centers
(not edges):
```python
pixel_offset = (pixel_size - grid_size) * 0.5
```

**Question:** Why do we add `pixel_size * 0.5` to the offset (embedded in the formula above)?
What would happen without it?

<details>
<summary>Discussion</summary>

Without the half-pixel offset, queries would be made from pixel corners instead of centers.
This causes the rendered image to be shifted by half a pixel. For visualization it's a
subtle difference, but for accurate SDF sampling it matters — you want each pixel's color
to represent the SDF value at its center.

The formula `(pixel_size - grid_size) * 0.5` combines two offsets:
- `-grid_size * 0.5`: centers the grid at the origin
- `+pixel_size * 0.5`: shifts to pixel centers
</details>

### Now write it

Combine everything: create a tetrahedron mesh and render a 2D color-mapped SDF slice.

This pulls together:
- 2D kernel launch (Exercise 2)
- Point queries and SDF (Exercises 3-4)
- Bourke color map (Exercise 5)

Use the tetrahedron from Exercise 4. Render a 200x200 pixel image of the XZ slice
at `y=0`, covering a 4x4 region centered at the origin.

In [ ]:
# Your code here.
#
# 1. Define your bourke_color_map @wp.func (copy from Exercise 5 or redefine).
#
# 2. Write a @wp.kernel called `render_sdf_slice` that takes:
#      pixel_size: float
#      pixel_offset: wp.vec2
#      mesh_id: wp.uint64
#      sdf_band_width: float
#      out_texture: wp.array(dtype=wp.vec3, ndim=2)
#
#    For each pixel (j, i = wp.tid()):
#      a. Compute 3D position: x = j*pixel_size + offset[0], y = 0, z = i*pixel_size + offset[1]
#      b. Query the mesh with wp.mesh_query_point()
#      c. Compute signed distance
#      d. Map to color with bourke_color_map using [-sdf_band_width/2, +sdf_band_width/2]
#      e. Store in out_texture[i, j]
#
# 3. Set up the grid:
#      grid_size = (4.0, 4.0)
#      pixel_size = 0.02  (-> 200x200 resolution)
#      sdf_band_width = 0.5 / pixel_size
#
# 4. Create the tetrahedron mesh, allocate the texture, launch the kernel.
#    Store the texture array in `sdf_texture`.


In [ ]:
# --- Assertions and visualization (do not modify) ---
tex = sdf_texture.numpy()
assert tex.shape[0] == 200 and tex.shape[1] == 200, f"Expected 200x200, got {tex.shape[:2]}"
# Center pixel should be inside the mesh (blue-ish, since SDF is negative there)
center = tex[100, 100]
print(f"Center pixel color: ({center[0]:.2f}, {center[1]:.2f}, {center[2]:.2f})")
# Corner pixel should be outside the mesh (red-ish, since SDF is positive)
corner = tex[0, 0]
print(f"Corner pixel color: ({corner[0]:.2f}, {corner[1]:.2f}, {corner[2]:.2f})")
print("Passed!")

# Visualize (optional — requires matplotlib)
try:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(tex)
    ax.set_title("SDF slice of tetrahedron (XZ plane)")
    ax.set_xlabel("X")
    ax.set_ylabel("Z")
    plt.show()
except ImportError:
    print("(matplotlib not available for visualization)")

---
## Exercise 7: Ray-Mesh Intersection (Raycasting)

### Think first

**Raycasting** shoots rays from a viewpoint through each pixel of an image and checks
what geometry the ray hits. It's the foundation of ray tracing renderers.

`wp.mesh_query_ray()` finds where a ray intersects a mesh. It takes:
- `mesh_id`: the mesh to test against
- `ray_origin`: where the ray starts (`wp.vec3`)
- `ray_dir`: the ray direction (`wp.vec3`, should be normalized)
- `max_t`: maximum distance to check

It returns a query result with:
- `result`: `bool` — whether the ray hit anything
- `normal`: `wp.vec3` — surface normal at the hit point
- `t`: `float` — distance from origin to hit point along the ray
- `face`, `u`, `v`: triangle and barycentric coordinates of the hit

A common visualization trick: map the surface normal to a color by
`color = normal * 0.5 + vec3(0.5)`. This shifts the `[-1, 1]` normal range to `[0, 1]`
RGB range.

**Question:** If a ray misses the mesh entirely, what color should the pixel be?

<details>
<summary>Discussion</summary>

Black `(0, 0, 0)` — the background color. When `query.result` is `False`, we simply
leave the pixel at its default/initialized value.
</details>

### Now write it

Write a raycasting kernel that renders the tetrahedron by shooting rays from a camera
and coloring pixels based on the surface normal at the hit point.

Camera setup:
- Origin at `(0, 0, 5)` (looking toward the origin)
- For each pixel, compute a direction toward the pixel's position on an image plane
  at `z=-1` (relative to the origin), then normalize it
- Use a 100x100 pixel grid covering a 3x3 region

In [ ]:
# Your code here.
#
# Write a @wp.kernel called `raycast_mesh` that takes:
#   pixel_size: float
#   pixel_offset: wp.vec2
#   mesh_id: wp.uint64
#   out_texture: wp.array(dtype=wp.vec3, ndim=2)
#
# For each pixel (j, i = wp.tid()):
#   1. Compute ray_origin = wp.vec3(0.0, 0.0, 5.0)
#   2. Compute ray_dir = wp.normalize(wp.vec3(
#        j * pixel_size + pixel_offset[0],
#        i * pixel_size + pixel_offset[1],
#        -1.0,
#      ))
#   3. Call wp.mesh_query_ray(mesh_id, ray_origin, ray_dir, 1e6)
#   4. If hit: color = query.normal * 0.5 + wp.vec3(0.5, 0.5, 0.5)
#   5. If miss: color = wp.vec3(0.0, 0.0, 0.0)
#   6. Store in out_texture[i, j]
#
# Set up:
#   grid_size = (3.0, 3.0), pixel_size = 0.03 (-> 100x100)
#   Create tetrahedron mesh, allocate texture, launch kernel.
#   Store the result in `ray_texture`.


In [ ]:
# --- Assertions and visualization (do not modify) ---
rt = ray_texture.numpy()
assert rt.shape == (100, 100, 3), f"Expected (100, 100, 3), got {rt.shape}"
# Center pixel should have hit the mesh (non-black)
center_pix = rt[50, 50]
print(f"Center pixel: ({center_pix[0]:.2f}, {center_pix[1]:.2f}, {center_pix[2]:.2f})")
assert np.any(center_pix > 0.01), "Center pixel should have hit the mesh (non-black)"
# Corner pixel should be black (ray missed)
corner_pix = rt[0, 0]
print(f"Corner pixel: ({corner_pix[0]:.2f}, {corner_pix[1]:.2f}, {corner_pix[2]:.2f})")
np.testing.assert_allclose(corner_pix, [0, 0, 0], atol=0.01, err_msg="Corner should be black (miss)")
print("Passed!")

# Visualize (optional — requires matplotlib)
try:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(np.clip(rt, 0, 1), origin="lower")
    ax.set_title("Raycasted tetrahedron (normal coloring)")
    plt.show()
except ImportError:
    print("(matplotlib not available for visualization)")

---
## Exercise 8: Break It — Mesh Winding Order

### Think first

The **winding order** of triangle vertices determines which side is the "front" (outside)
and which is the "back" (inside). This directly affects `query.sign` in SDF computations
and `query.normal` in raycasting.

If you reverse the winding order of all triangles (swap two vertices in each face),
the normals flip, and inside/outside are swapped.

**Question:** If you flip the winding order of the tetrahedron from Exercise 4,
what would happen to the SDF value at the origin? What about the ray-traced normals?

<details>
<summary>Discussion</summary>

The SDF at the origin would become **positive** (the mesh now thinks the origin is
"outside"). The ray-traced normals would point **inward** instead of outward, producing
different (darker) colors since the normal-to-color mapping would give values closer
to 0.
</details>

### Now try it

Create a tetrahedron with **reversed** winding order. Compute the SDF at the origin and
verify that the sign has flipped compared to Exercise 4.

In [ ]:
# Your code here.
#
# The original faces were: [0,1,2], [0,2,3], [0,3,1], [1,3,2]
# Reverse each by swapping the last two vertices:
#   [0,2,1], [0,3,2], [0,1,3], [1,2,3]
#
# Create the flipped mesh, query the SDF at the origin.
# Store the result in `flipped_sdf_val` (a single float).


In [ ]:
# --- Assertions (do not modify) ---
print(f"Flipped SDF at origin: {flipped_sdf_val}")
assert flipped_sdf_val > 0.0, f"Flipped winding should make origin 'outside' (positive SDF), got {flipped_sdf_val}"
print("Passed!")

---
## Exercise 9: Break It — Ray Direction

Take your working raycast kernel from Exercise 7 and experiment with these changes
**one at a time**. Predict what will happen, then run it.

1. **Don't normalize** the ray direction — remove the `wp.normalize()` call
2. **Move the camera inside** the mesh — set `ray_origin = wp.vec3(0, 0, 0)`
3. **Point the camera away** — negate the z component of the direction

<details>
<summary>Discussion</summary>

1. **Unnormalized rays:** The rendering still "works" visually, but `query.t` now
   represents distance in units of the unnormalized direction vector, not world units.
   For normal coloring this doesn't matter, but for depth calculations it would be wrong.

2. **Camera inside:** You see the *inside* of the tetrahedron faces (backfaces). The
   normals point away from you, so the normal-to-color mapping gives dark/different colors.

3. **Camera facing away:** The image is entirely black — no rays hit the mesh since
   they're pointing in the wrong direction.
</details>

In [ ]:
# Experiment here: copy your raycast setup from Exercise 7 and try the modifications.

---
## Exercise 10: Putting It Together — Depth Rendering

Write a raycasting kernel that renders a **depth image** instead of normals.
The depth is `query.t` — the distance from the camera to the hit point along the ray.

Map depth to brightness: closer points should be **brighter** (white), and farther points
**darker** (black). A simple mapping:
```python
brightness = 1.0 - (t - min_depth) / (max_depth - min_depth)
color = wp.vec3(brightness, brightness, brightness)
```

Use `min_depth = 3.5` and `max_depth = 6.0` for the tetrahedron with a camera at `z=5`.
Clamp brightness to `[0, 1]`.

Render a 100x100 image. Pixels that miss the mesh should be black.

In [ ]:
# Your code here.
#
# Write a @wp.kernel called `raycast_depth` similar to Exercise 7, but:
#   - Store depth-based brightness instead of normal-based color
#   - Takes additional inputs: min_depth (float) and max_depth (float)
#
# Create the tetrahedron mesh, set up the grid, launch the kernel.
# Store the result in `depth_texture`.


In [ ]:
# --- Assertions and visualization (do not modify) ---
dt = depth_texture.numpy()
assert dt.shape == (100, 100, 3), f"Expected (100, 100, 3), got {dt.shape}"
# Center pixel should be bright (close to camera)
center_depth = dt[50, 50]
print(f"Center pixel brightness: {center_depth[0]:.2f}")
assert center_depth[0] > 0.3, "Center should be somewhat bright (mesh is close)"
# Corner should be black
corner_depth = dt[0, 0]
np.testing.assert_allclose(corner_depth, [0, 0, 0], atol=0.01)
print("Passed!")

# Visualize (optional)
try:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(np.clip(dt, 0, 1), origin="lower")
    ax.set_title("Depth rendering of tetrahedron")
    plt.show()
except ImportError:
    print("(matplotlib not available for visualization)")